In [ ]:
import cv2
import csv
import numpy as np
import os
import datetime as dt
from tqdm import tqdm
import moviepy.editor as mp

In [ ]:
######################
#視線データの読み込み
######################

#視線生データ
gaze_data = "output/output_gaze.csv"

# csvを一行づつ読み込み視線データをdictに格納
gp_dict={} #注視点格納用dictionary
with open(gaze_data) as f: #生データ
    for df in csv.reader(f):
        if df == []:
            continue
        row = [df[0],df[1],df[2],df[3],df[4]
        #データの欠損状況で場合分け
        if row[1] != 'nan' and row[3] != 'nan':
            x = (float(row[1])+float(row[3]))/2
            y = (float(row[2])+float(row[4]))/2
        elif row[1] != 'nan' and row[3] == 'nan':
            x = float(row[1])
            y = float(row[2])
        elif row[1] == 'nan' and row[3] != 'nan':
            x = float(row[3])
            y = float(row[4])
        else: #全欠損の場合は−１とする
            x = -1
            y = -1
        #時刻が'17:53:09'のように秒以下が無い(.000000になっている)時は.000000を文字列に追加。
        if len(row[0]) == 19:
            row[0] = row[0] + '.000000'
        gaze_time = dt.datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S.%f') #文字列を時刻に変換 #https://techacademy.jp/magazine/23375
        gp_dict[gaze_time] = [x,y] #時刻をkey、注視点座標をvalueに


In [ ]:
######################
#画面録画の動画読み込み
######################

# 画面録画の動画（インプット）
video = "output/screen_record.mp4"

# 動画の開始時刻 （XXXに時刻を入力）
start_time = dt.datetime.strptime('XXXX-XX-XX XX:XX:XX.XXXXXX', '%Y-%m-%d %H:%M:%S.%f')

#動画の読み込み
cap = cv2.VideoCapture(video)

#動画の情報
video_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))# 幅
video_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))# 高さ
count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))# 総フレーム数
fps = cap.get(cv2.CAP_PROP_FPS)# フレームレート

In [ ]:
######################
#動画と注視点を時刻で突き合わせて出力
######################

# 注視点を追加した出力動画（アウトプット）
result = "output/gaze_movement.mp4"

#出力動画の形式
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
output = cv2.VideoWriter( result, fourcc, 30, (int(video_w), int(video_h)))

# 描画用パラメータ
r = 30 #円の半径
screen_w = 1920
screen_h = 1080

#動画への注視点の描画
video_time = start_time
for i in range(count):
    ret, frame_ = cap.read()# 動画から1フレーム取得する
    if ret:
        # リサイズ
        frame = cv2.resize(frame_, (int(video_w), int(video_h)))
        
        # video_timeに最も近い時刻の注視点(x,y)を取得
        gaze_point = gp_dict[min(gp_dict.keys(), key = lambda k: abs(k - video_time))]
        
        if gaze_point[0] != -1: #視線位置が欠損していなければ注視点を描画
            x = int(screen_w * zoom * gaze_point[0])
            y = int(screen_h * zoom *  gaze_point[1])
            cv2.circle(frame,(x, y), r, (0, 0, 255), int(5 * zoom))

        # 時刻を描画
        cv2.putText(frame, str(video_time), (0, 30), cv２.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1, cv２.LINE_AA)
        
        #フレームを.mp4に出力
        output.write(frame) 
        #1フレーム分時刻を進める
        video_time = video_time + dt.timedelta(microseconds = 1000000/fps)
            
    else:
        break  
        
    if cv2.waitKey(1) & 0xFF == ord('q'): #[q]キーを押して処理を中断。
        break
            
cap.release()